In [19]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns
import lightgbm as lgb
from sklearn.model_selection import KFold
import warnings
import time
import sys
import datetime
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import log_loss
warnings.simplefilter(action='ignore', category=FutureWarning)
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows',300)
from scipy.stats import mode
from scipy import stats
import feather
import re
import warnings
import random

from IPython.display import clear_output

In [2]:
target = pd.read_csv('target2',header=None,index_col=0)

In [3]:
train = feather.read_dataframe('best_from_recent_groupped_train')
test = feather.read_dataframe('best_from_recent_groupped_test')

In [4]:
new_test_cols = feather.read_dataframe('new_test_columns_with_promos_fixed')

train = pd.merge(train,new_test_cols,on='card_id',how='left')
test = pd.merge(test,new_test_cols,on='card_id',how='left')

new_test_cols =  feather.read_dataframe('gropped_columns_per_month_to_last_ver2')
train = pd.merge(train,new_test_cols,on='card_id',how='left')
test = pd.merge(test,new_test_cols,on='card_id',how='left')

train_out = feather.read_dataframe('train_months_outliner_percent')
test_out = feather.read_dataframe('test_months_outliner_percent')

train = pd.merge(train,train_out,on='card_id',how='left')
test = pd.merge(test,test_out,on='card_id',how='left')

In [32]:
target = target < -30

In [ ]:
for z_test in sorted(list(set(all_cols.importance.astype(np.int16)))):
    
    best_cols =list(all_cols[all_cols.importance > z_test].index)
    
    if 'best_cols' in locals():
        features = [c for c in best_cols if c not in ['card_id', 'first_active_month','last_active_month']]
    else:
        features = [c for c in train.columns if c not in ['card_id', 'first_active_month','last_active_month']]
    
    
    additional_cats= [x for x in features if re.search('[th]_cat1_[^s]',x)]
    categorical_feats = [x for x in features if (re.search('modeplus',x) or re.search('feature_',x))] + additional_cats 
    categorical_feats = list(set(categorical_feats)) + [x for x in ['last4_months_cat','client_wave','during_promo'] if x in features] #first_active_month_coded
    warnings.filterwarnings("ignore")

    param = {
            'num_leaves': 111, #checked
             'num_threads': 4,
             'max_bin': 170, #checked 170
             'min_data_in_leaf': 100, #checked 100
             'objective':'binary',
             'max_depth': 9, #checked
             'tree_learner':'data',
             'learning_rate': 0.07,
             "boosting": "gbdt", #dart, gbdt
             "feature_fraction": 0.58, #checked 0.58
             "bagging_freq": 7, #5
             "bagging_fraction": 0.97, #checked
             "bagging_seed": 11,
             "lambda_l1": 0.23, #checked 26
    #            'lambda_l2': 0.3, #checked
             "random_state": 77,
             "verbosity": -1,
             "device":"gpu",
             'gpu_device_id':0,
             'gpu_platform_id':1,
             'gpu_use_dp':True,
             'is_unbalanced':True,
            }


    folds = KFold(n_splits=5, shuffle=True, random_state=15)
    oof = np.zeros(len(train))
    predictions = np.zeros(len(test))
    start = time.time()
    feature_importance_df = pd.DataFrame()

    for fold_, (trn_idx, val_idx) in enumerate(folds.split(train.values, target.values)):
        print("fold n°{}".format(fold_))

        trn_data = lgb.Dataset(train.iloc[trn_idx][features],
                               label=target.iloc[trn_idx],
                               categorical_feature=categorical_feats
                              )
        val_data = lgb.Dataset(train.iloc[val_idx][features],
                               label=target.iloc[val_idx],
                               categorical_feature=categorical_feats
                              )

        clf = lgb.train(param, 
                        trn_data,10000, 
                        valid_sets = [trn_data, val_data], 
                        verbose_eval=250, early_stopping_rounds = 250)

        oof[val_idx] = clf.predict(train.iloc[val_idx][features], num_iteration=clf.best_iteration)

        fold_importance_df = pd.DataFrame()
        fold_importance_df["feature"] = features
        fold_importance_df["importance"] = clf.feature_importance()
        fold_importance_df["fold"] = fold_ + 1
        feature_importance_df = pd.concat([feature_importance_df, fold_importance_df], axis=0)

        predictions += clf.predict(test[features], num_iteration=clf.best_iteration) / folds.n_splits
        
    errr = log_loss(target,oof)
    print("CV score: {:<8.5f}\n".format(errr))

fold n°0
Training until validation scores don't improve for 250 rounds.
[250]	training's binary_logloss: 0.00296646	valid_1's binary_logloss: 0.055568
Early stopping, best iteration is:
[38]	training's binary_logloss: 0.0257952	valid_1's binary_logloss: 0.0463549
fold n°1
Training until validation scores don't improve for 250 rounds.
[250]	training's binary_logloss: 0.00300673	valid_1's binary_logloss: 0.0527288
Early stopping, best iteration is:
[49]	training's binary_logloss: 0.0230959	valid_1's binary_logloss: 0.0446044
fold n°2
Training until validation scores don't improve for 250 rounds.
[250]	training's binary_logloss: 0.00305759	valid_1's binary_logloss: 0.0506542
Early stopping, best iteration is:
[57]	training's binary_logloss: 0.0218491	valid_1's binary_logloss: 0.0432052
fold n°3
Training until validation scores don't improve for 250 rounds.
[250]	training's binary_logloss: 0.00307349	valid_1's binary_logloss: 0.0580806
Early stopping, best iteration is:
[41]	training's bin

Early stopping, best iteration is:
[49]	training's binary_logloss: 0.0229429	valid_1's binary_logloss: 0.045992
fold n°1
Training until validation scores don't improve for 250 rounds.
[250]	training's binary_logloss: 0.00290114	valid_1's binary_logloss: 0.0524865
Early stopping, best iteration is:
[58]	training's binary_logloss: 0.0212701	valid_1's binary_logloss: 0.0442935
fold n°2
Training until validation scores don't improve for 250 rounds.
[250]	training's binary_logloss: 0.00321262	valid_1's binary_logloss: 0.0508134
Early stopping, best iteration is:
[65]	training's binary_logloss: 0.0206433	valid_1's binary_logloss: 0.0437884
fold n°3
Training until validation scores don't improve for 250 rounds.
[250]	training's binary_logloss: 0.0029676	valid_1's binary_logloss: 0.0596754
Early stopping, best iteration is:
[46]	training's binary_logloss: 0.0234631	valid_1's binary_logloss: 0.049886
fold n°4
Training until validation scores don't improve for 250 rounds.
[250]	training's binary

[250]	training's binary_logloss: 0.00300166	valid_1's binary_logloss: 0.0536115
Early stopping, best iteration is:
[51]	training's binary_logloss: 0.0227176	valid_1's binary_logloss: 0.0449946
fold n°2
Training until validation scores don't improve for 250 rounds.
[250]	training's binary_logloss: 0.00299218	valid_1's binary_logloss: 0.0509824
Early stopping, best iteration is:
[45]	training's binary_logloss: 0.0239221	valid_1's binary_logloss: 0.0432242
fold n°3
Training until validation scores don't improve for 250 rounds.
[250]	training's binary_logloss: 0.00267875	valid_1's binary_logloss: 0.0598651
Early stopping, best iteration is:
[40]	training's binary_logloss: 0.0248221	valid_1's binary_logloss: 0.0494592
fold n°4
Training until validation scores don't improve for 250 rounds.
[250]	training's binary_logloss: 0.00282033	valid_1's binary_logloss: 0.0518218
Early stopping, best iteration is:
[66]	training's binary_logloss: 0.0193704	valid_1's binary_logloss: 0.0441101
CV score: 0.

fold n°2
Training until validation scores don't improve for 250 rounds.
[250]	training's binary_logloss: 0.00306103	valid_1's binary_logloss: 0.0509447
Early stopping, best iteration is:
[53]	training's binary_logloss: 0.0227006	valid_1's binary_logloss: 0.0438909
fold n°3
Training until validation scores don't improve for 250 rounds.
[250]	training's binary_logloss: 0.00288538	valid_1's binary_logloss: 0.058842
Early stopping, best iteration is:
[46]	training's binary_logloss: 0.0232062	valid_1's binary_logloss: 0.0492536
fold n°4
Training until validation scores don't improve for 250 rounds.
[250]	training's binary_logloss: 0.00294701	valid_1's binary_logloss: 0.0518657
Early stopping, best iteration is:
[39]	training's binary_logloss: 0.0259248	valid_1's binary_logloss: 0.0439441
CV score: 0.04558 

fold n°0
Training until validation scores don't improve for 250 rounds.
[250]	training's binary_logloss: 0.00297876	valid_1's binary_logloss: 0.0554691
Early stopping, best iteration is:

Early stopping, best iteration is:
[50]	training's binary_logloss: 0.0240915	valid_1's binary_logloss: 0.0437083
fold n°3
Training until validation scores don't improve for 250 rounds.
[250]	training's binary_logloss: 0.00311028	valid_1's binary_logloss: 0.0591072
Early stopping, best iteration is:
[40]	training's binary_logloss: 0.0252106	valid_1's binary_logloss: 0.0493981
fold n°4
Training until validation scores don't improve for 250 rounds.
[250]	training's binary_logloss: 0.00321123	valid_1's binary_logloss: 0.0515917
Early stopping, best iteration is:
[46]	training's binary_logloss: 0.0243565	valid_1's binary_logloss: 0.0441121
CV score: 0.04559 

fold n°0
Training until validation scores don't improve for 250 rounds.
[250]	training's binary_logloss: 0.00313812	valid_1's binary_logloss: 0.0548002
Early stopping, best iteration is:
[47]	training's binary_logloss: 0.0237755	valid_1's binary_logloss: 0.0459374
fold n°1
Training until validation scores don't improve for 250 rounds.


In [17]:
cols = (feature_importance_df[["feature", "importance"]]
        .groupby("feature")
        .mean()
        .sort_values(by="importance", ascending=False)[:1000].index)

best_features = feature_importance_df.loc[feature_importance_df.feature.isin(cols)]

In [18]:
all_cols = pd.DataFrame(best_features.groupby(['feature'])['importance'].mean()).sort_values(by='importance')
all_cols

,importance
feature,
proc_purchase_date_day_diff_in_days_min_min,0.0
installment_size__999,0.0
installment_size__11,0.0
proc_NNN_installments_mean_mean,0.0
first_purch_during_activation,0.4
installment_size__9,0.6
still_active_card,0.6
installment_size__7,1.0
proc_installments_min_min,2.4
